<a href="https://colab.research.google.com/github/ziyacanyldz/inme-tahmini/blob/main/D2_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd ; import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline 

In [ ]:
from google.colab import files
  
  
uploaded = files.upload()

Saving inme_veriseti.csv to inme_veriseti.csv


In [ ]:
import io
veri = pd.read_csv(io.BytesIO(uploaded['inme_veriseti.csv']))
veri

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [ ]:
del veri["id"]

In [ ]:
veri.dropna(subset=["bmi"],inplace=True)

In [ ]:
veri['gender'] = veri['gender'].replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)
veri['Residence_type'] = veri['Residence_type'].replace({'Rural':0,'Urban':1}).astype(np.uint8)
veri['work_type'] = veri['work_type'].replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':-1,'Never_worked':-2}).astype(np.uint8)
veri['ever_married'] = veri['ever_married'].replace({'Yes':1, 'No':0}).astype(np.uint8)
veri['smoking_status'] = veri['smoking_status'].replace({'never smoked':0,'Unknown':1,'formerly smoked':2,'smokes':-1}).astype(np.uint8)

In [ ]:
veri

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,1,0,1,228.69,36.6,2,1
2,0,80.0,0,1,1,0,0,105.92,32.5,0,1
3,1,49.0,0,0,1,0,1,171.23,34.4,255,1
4,1,79.0,1,0,1,1,0,174.12,24.0,0,1
5,0,81.0,0,0,1,0,1,186.21,29.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,1,13.0,0,0,0,255,0,103.08,18.6,1,0
5106,1,81.0,0,0,1,1,1,125.20,40.0,0,0
5107,1,35.0,0,0,1,1,0,82.99,30.6,0,0
5108,0,51.0,0,0,1,0,0,166.29,25.6,2,0


In [ ]:
x = veri.drop('stroke' , axis = 1)

In [ ]:
y = veri['stroke']

In [ ]:
from imblearn.over_sampling import SMOTE 

In [ ]:
oversample = SMOTE()
x_resh, y_resh = oversample.fit_resample(x , y.ravel())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test =  train_test_split(x_resh, y_resh , test_size=0.20 ,random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import  SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def svc_param_selection(X , Y , nfolds):
  Cler = [0.0001,0.001,0.01,0.1,0.5,1,5,10,100,1000]
  gammalar = [0.001 , 0.01 , 0.1 , 1]
  param_grid = {"C":Cler , "gamma":gammalar  }
  grid_search = GridSearchCV(SVC() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
svc_param_selection(x_train , y_train , 4)

{'C': 5, 'gamma': 0.1}


In [ ]:
def XgBoost_param_selection(X , Y , nfolds):
  max_depth = range(2,10,1)
  n_estimators = range(60,220,40)
  learning_rate = [0.1,0.01,0.05]
  param_grid = {"max_depth":max_depth , "n_estimators":n_estimators , "learning_rate" :learning_rate  }
  grid_search = GridSearchCV(XGBClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
XgBoost_param_selection(x_train , y_train , 4)

{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 180}


In [ ]:
def KNeighbors_param_selection(X , Y , nfolds):
  N_neighbors = [2,4,8,16,32]
  p = [2,3]
  param_grid = {"n_neighbors":N_neighbors , "p":p  }
  grid_search = GridSearchCV(KNeighborsClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
KNeighbors_param_selection(x_train , y_train , 4)

{'n_neighbors': 2, 'p': 2}


In [ ]:
def LogisticRegression_param_selection(X , Y , nfolds):
  Penalty = ["l1" , "l2"]
  C = np.logspace(-4, 4, 20)
  Max_iter = [ 1000,1500]
  param_grid = {"penalty":Penalty , "C":C , "max_iter":Max_iter  }
  grid_search = GridSearchCV(LogisticRegression() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
LogisticRegression_param_selection(x_train , y_train , 4)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
160 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Sol

{'C': 0.23357214690901212, 'max_iter': 1000, 'penalty': 'l2'}


In [ ]:
def RandomForestClassifier_param_selection(X , Y , nfolds):
  N_estimators = [120,500,700]
  Max_depth = [5,15,30,None]
  Min_samples_split = [1,3,20]
  Min_samples_leaf =[1,5,10]
  Max_features = ["auto" , "sqrt" ]
  param_grid = {"n_estimators":N_estimators,"max_depth":Max_depth,"min_samples_split":Min_samples_split,"min_samples_leaf":Min_samples_leaf,"max_features":Max_features }
  grid_search = GridSearchCV(RandomForestClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
RandomForestClassifier_param_selection(x_train , y_train , 4)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
288 fits failed out of a total of 864.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
288 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 459, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/p

{'max_depth': 30, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 700}


In [ ]:
def BernoulliNB_param_selection(X , Y , nfolds):
  Alpha = [0.01, 0.1, 0.5, 1.0, 10.0]
  param_grid = {"alpha":Alpha}
  grid_search = GridSearchCV(BernoulliNB() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
BernoulliNB_param_selection(x_train , y_train , 4)

{'alpha': 10.0}


In [ ]:
def GaussianNB_param_selection(X , Y , nfolds):
  var_smoothing = np.logspace(0,-9, num=100)
  param_grid = {"var_smoothing":var_smoothing}
  grid_search = GridSearchCV(GaussianNB() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
GaussianNB_param_selection(x_train , y_train , 4)

{'var_smoothing': 0.0012328467394420659}


In [ ]:
def Mlp_param_selection(X , Y , nfolds):
  hidden_layer_sizes = [(50,50,50), (50,100,50), (100,)]
  activation = ['tanh', 'relu']
  solver = ['sgd', 'adam']
  alpha =[0.0001, 0.05]
  learning_rate =['constant','adaptive']
  Max_iter = [ 500,800]
  param_grid = {"hidden_layer_sizes":hidden_layer_sizes,"activation":activation,"solver":solver,"alpha":alpha,"learning_rate":learning_rate ,"max_iter":Max_iter }
  grid_search = GridSearchCV(MLPClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
Mlp_param_selection(x_train , y_train , 4)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'max_iter': 500, 'solver': 'adam'}


In [ ]:
def GradientBoosting_param_selection(X , Y , nfolds):
  n_estimators = [10, 100, 500]
  learning_rate = [ 0.001, 0.01, 0.1, 1.0]
  subsample = [0.5, 1.0]
  max_depth =[3, 7, 9]
  loss = ["deviance","exponential"]
  param_grid = {"n_estimators":n_estimators,"learning_rate":learning_rate,"subsample":subsample,"max_depth":max_depth , "loss":loss }
  grid_search = GridSearchCV(GradientBoostingClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
GradientBoosting_param_selection(x_train , y_train , 4)

{'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 9, 'n_estimators': 500, 'subsample': 0.5}


In [ ]:
def DecisionTree_param_selection(X , Y , nfolds):
  criterion = ['gini','entropy']
  max_depth = [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
  param_grid = {"criterion":criterion,"max_depth":max_depth }
  grid_search = GridSearchCV(DecisionTreeClassifier() , param_grid , cv=nfolds)
  grid_search.fit(X,Y)
  return print(grid_search.best_params_)

In [ ]:
DecisionTree_param_selection(x_train , y_train , 4)

{'criterion': 'entropy', 'max_depth': 70}
